### Fine tune LLMs using LLaMA Factory

- https://github.com/hiyouga/LLaMA-Factory
- Fine tune mistral 7b instruction version

### Game plan
- mistral > fine tune > Docker natural language commands dataset > RAG

- Q: How to delete a docker container.
- A: docker rm -f [container-name]

### Datasets
 - Data on huggingface
- https://huggingface.co/datasets/MattCoddity/dockerNLcommands

- Use Gradio UI
- https://www.gradio.app/

### Setup libraries

In [1]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git

fatal: destination path 'LLaMA-Factory' already exists and is not an empty directory.


In [2]:
%cd LLaMA-Factory/

/content/LLaMA-Factory


In [3]:
!pwd

/content/LLaMA-Factory


In [6]:
!pip install -r requirements.txt

  Using cached peft-0.12.0-py3-none-any.whl.metadata (13 kB)
Using cached peft-0.12.0-py3-none-any.whl (296 kB)
  Attempting uninstall: peft
    Found existing installation: peft 0.15.1
    Uninstalling peft-0.15.1:
      Successfully uninstalled peft-0.15.1


In [8]:
!pip install bitsandbytes

In [23]:
#!pip install peft>=0.12.0,<=0.15.1

/bin/bash: line 1: =0.15.1: No such file or directory


In [7]:
!pip install llamafactory

In [27]:
!DISABLE_VERSION_CHECK=1

### Changes in LLaMA Factory code

In [14]:
!GRADIO_SHARE=1 llamafactory-cli webui

2025-04-21 19:35:24.530479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745264124.755814   55305 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745264124.814587   55305 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-21 19:35:25.277611: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://7a61a08cb61cdb6add.gradio.live

This sh

In [17]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write

In [18]:
!huggingface-cli upload RickyRubini/my-llama-tuned /content/LLaMA-Factory/saves/LLaVA-NeXT-Mistral-7B-Chat/lora/train_2025-04-21-17-35-28

Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
Start hashing 20 files.
Finished hashing 20 files.
tokenizer.model:   0% 0.00/493k [00:00<?, ?B/s]
adapter_model.safetensors:   0% 0.00/84.0M [00:00<?, ?B/s]

Upload 3 LFS files:   0% 0/3 [00:00<?, ?it/s]


training_args.bin:   0% 0.00/5.69k [00:00<?, ?B/s]
training_args.bin: 100% 5.69k/5.69k [00:00<00:00, 31.8kB/s]

adapter_model.safetensors:   9% 7.37M/84.0M [00:00<00:02, 30.4MB/s]
tokenizer.model: 100% 493k/493k [00:00<00:00, 1.05MB/s]

adapter_model.safetensors:  19% 16.0M/84.0M [00:00<00:04, 14.7MB/s]
adapter_model.safetensors:  29% 24.0M/84.0M [00:01<00:02, 25.1MB/s]
adapter_model.safetensors:  33% 27.9M/84.0M [00:01<00:02, 21.2MB/s]
adapter_model.safetensors:  37% 31.3M/84.0M [00:01<00:02, 23.0MB/s]
adapter_model.safetensors:  41% 34.4M/84.0M [00:01<00:03, 14.5MB/s]
adapter_model.safetensors:  55% 46.3M/84.0M [0

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the base model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
base_model = AutoModelForCausalLM.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
model = PeftModel.from_pretrained(base_model, "RickyRubini/my-llama-tuned")

query_to_llm = "It was originally priced at $200 but it's on sale for 20% off. Can you tell me how much it will cost after the discount?"

# Generate an answer from the model
inputs = tokenizer.encode(query_to_llm, return_tensors="pt")
outputs = model.generate(inputs)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)